<참조 : https://www.tensorflow.org/tutorials/keras/save_and_load>

## 모델 저장과 복원 
  - 훈련하는 도중이나 훈련이 끝난 후에 모델을 저장
    - 훈련이 중지된 지점부터 다시 훈련할 수 있습니다
    - 다른 작업자와 공유할 수 있구 작업을 재현할 수 있습니다  
      
  - 모델을 만드는 코드
  - 모델의 훈련된 가중치 / 파라미터

### 01. 설정과 임포트
  - 모델을 저장하기 위한 포맷 지원 라이브러리 설치

In [1]:
!pip install -q pyyaml h5py

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


####  예제 데이터셋 받기

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]


train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 02. 모델 정의

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 03. 훈련하는 동안 체크포인트 저장하기
  - 훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장
  - 모델을 재사용하거나 이어서 훈련을 진행할 수 있습니다

### 03. 01. 체크포인트 콜백 사용하기
  - 훈련하는 동안 가중치를 저장하기 위해 ModelCheckpoint 콜백을 만듭니다

In [5]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

# 새로운 콜랩으로 모델 훈련하기
model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])    # 콜백을 훈련에 전달합니다

Epoch 1/10
32/32 [==============================] - 6s 48ms/step - loss: 1.2196 - accuracy: 0.6430 - val_loss: 0.7373 - val_accuracy: 0.7820

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 1s 25ms/step - loss: 0.4517 - accuracy: 0.8760 - val_loss: 0.6355 - val_accuracy: 0.7910

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 1s 24ms/step - loss: 0.3084 - accuracy: 0.9120 - val_loss: 0.4975 - val_accuracy: 0.8400

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 1s 23ms/step - loss: 0.2155 - accuracy: 0.9490 - val_loss: 0.4705 - val_accuracy: 0.8480

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 1s 24ms/step - loss: 0.1652 - accuracy: 0.9600 - val_loss: 0.4337 - val_accuracy: 0.8570

Epoch 00005: saving model to training_1\cp.ckpt
Epoch 6/10
32/32 [==============================] 

In [7]:
ls {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F2C2-2A72

 C:\Users\jjanh_000\python\deepLearning\tensorflowTutorial\training_1 디렉터리

2021-07-05  오전 10:24    <DIR>          .
2021-07-05  오전 10:24    <DIR>          ..
2021-07-05  오전 10:24                71 checkpoint
2021-07-05  오전 10:24         4,886,673 cp.ckpt.data-00000-of-00001
2021-07-05  오전 10:24             1,222 cp.ckpt.index
               3개 파일           4,887,966 바이트
               2개 디렉터리  831,197,618,176 바이트 남음


In [9]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('훈련되지 않은 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 1s - loss: 2.3505 - accuracy: 0.1060
훈련되지 않은 모델의 정확도: 10.60%


In [10]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4029 - accuracy: 0.8700
복원된 모델의 정확도: 87.00%


### 03. 02. 체크포인트 콜백 매개변수
  - 새로운 모델을 훈련하고 에포크마다 고유한 이름으로 체크포인트를 저장

In [14]:
# 파일 이름에 에포크 번호를 포함시킵니다('str.format' 포뱃)
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    preiod=5
)

# 새로운 모델 객체를 만듭니다
model = create_model()

# 'checkpoint_path' 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images,
         train_labels,
         epochs=50,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)


Epoch 00001: saving model to training_2\cp-0001.ckpt

Epoch 00002: saving model to training_2\cp-0002.ckpt

Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00004: saving model to training_2\cp-0004.ckpt

Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00006: saving model to training_2\cp-0006.ckpt

Epoch 00007: saving model to training_2\cp-0007.ckpt

Epoch 00008: saving model to training_2\cp-0008.ckpt

Epoch 00009: saving model to training_2\cp-0009.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00011: saving model to training_2\cp-0011.ckpt

Epoch 00012: saving model to training_2\cp-0012.ckpt

Epoch 00013: saving model to training_2\cp-0013.ckpt

Epoch 00014: saving model to training_2\cp-0014.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00016: saving model to training_2\cp-0016.ckpt

Epoch 00017: saving model to training_2\cp-0017.ckpt

Epoch 00018: saving model to training_2\cp-0018.ckpt

Epoch 00019: saving model t

In [15]:
ls {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F2C2-2A72

 C:\Users\jjanh_000\python\deepLearning\tensorflowTutorial\training_2 디렉터리

2021-07-05  오전 10:37    <DIR>          .
2021-07-05  오전 10:37    <DIR>          ..
2021-07-05  오전 10:37                81 checkpoint
2021-07-05  오전 10:35         1,628,730 cp-0000.ckpt.data-00000-of-00001
2021-07-05  오전 10:35               402 cp-0000.ckpt.index
2021-07-05  오전 10:36         4,886,697 cp-0001.ckpt.data-00000-of-00001
2021-07-05  오전 10:36             1,222 cp-0001.ckpt.index
2021-07-05  오전 10:36         4,886,697 cp-0002.ckpt.data-00000-of-00001
2021-07-05  오전 10:36             1,222 cp-0002.ckpt.index
2021-07-05  오전 10:36         4,886,697 cp-0003.ckpt.data-00000-of-00001
2021-07-05  오전 10:36             1,222 cp-0003.ckpt.index
2021-07-05  오전 10:36         4,886,697 cp-0004.ckpt.data-00000-of-00001
2021-07-05  오전 10:36             1,222 cp-0004.ckpt.index
2021-07-05  오전 10:36         4,886,697 cp-0005.ckpt.data-00000-of-00001
2021-07-05  오전 10:36   

In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

#### 텐서플로는 기본적으로 최근 체크포인트 5개만 저장합니다
  - 모델을 초기화하고 체크포인트를 로드하여 테스트
 

In [17]:
# 새로운 모델 객체를 만듭니다.
model =create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 1s - loss: 0.4808 - accuracy: 0.8710
복원된 모델의 정확도: 87.10%


### 03. 03. 수동으로 가중치 저장하기
  - Model.save_weights 메서드 사용
  - tf.keras : save_weights는 ".ckpt" 확장자를 가진 텐서플로 체크포인트 포맷을 사용
  - ".h5" 확장자의 HDF5로 저장하는 것은 별도 : https://www.tensorflow.org/guide/keras/save_and_serialize#weights_only_saving_in_savedmodel_format

In [18]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다. 
loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 1s - loss: 0.4808 - accuracy: 0.8710
복원된 모델의 정확도: 87.10%


### 04. 전체 모델 저장하기
  - model.save 메서드 호출
    - 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다
    - 모델을 저장하기 때문에 원본 파이썬 코드가 없ㅇ도 사용할 수 있습니다
    - 옵티마이저 상태가 복원되므로 중지한 시점에서 다시 훈련을 재개할 수 있습니다  
      
        
  - SavedModel / HDF5 : 두 개 모델로 저장할 수 있습니다  
    
      
  - 전체 모델 저장
    - TensorFlow.js로 모델을 로드
    - 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다 : Saved Model / HDF5
    - 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다
   

#### 04. 01. SavedModel 포맷
  - 모델을 직렬화 하는 다른 방법
  - tf.keras.models.load_model 로 복원할 수 있습니다 / 텐서플로 서빙과 호환됩니다
  - https://www.tensorflow.org/guide/saved_model

In [19]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 2s 15ms/step - loss: 1.1288 - accuracy: 0.6850
Epoch 2/5
32/32 [==============================] - 0s 15ms/step - loss: 0.4239 - accuracy: 0.8850 0s - loss: 0.4339 - accuracy
Epoch 3/5
32/32 [==============================] - 1s 16ms/step - loss: 0.2805 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 1s 16ms/step - loss: 0.2077 - accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 1s 17ms/step - loss: 0.1499 - accuracy: 0.9700
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [25]:
# my_model 디렉토리
print(os.listdir("./saved_model"))
# ls ./saved_model

# assests 폴더, saved_model.pb, variables 폴더
print(os.listdir("./saved_model/my_model"))
# ls {./saved_model/my_model}

['my_model']
['assets', 'keras_metadata.pb', 'saved_model.pb', 'variables']


In [27]:
# 저장된 모델로부터 새로운 케라스 모델을 로드합니다
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 복원된 모델 : 원본 모델과 동일한 매개변수로 컴파일되어 있습니다
#### 이 모델을 평가하고 예측에 사용

In [28]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)

print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 1s - loss: 0.4169 - accuracy: 0.8610
복원된 모델의 정확도: 86.10%
(1000, 10)


#### 04. 02. HDF5 파일로 저장하기

In [29]:
# 새로운 모델 객체를 만들고 훈련합니다
model_hdf5 = create_model()
model_hdf5.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 2s 17ms/step - loss: 1.0890 - accuracy: 0.6950 0s - loss: 1.1394 - accuracy: 0.
Epoch 2/5
32/32 [==============================] - 1s 20ms/step - loss: 0.4262 - accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 1s 24ms/step - loss: 0.2815 - accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 1s 28ms/step - loss: 0.2009 - accuracy: 0.9510 0s - loss: 0.1984 - accuracy
Epoch 5/5
32/32 [==============================] - 1s 19ms/step - loss: 0.1460 - accuracy: 0.9730


In [30]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model_hdf5 = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model_hdf5.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model_hdf5.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 1s - loss: 0.4169 - accuracy: 0.8610
복원된 모델의 정확도: 86.10%


## 케라스 모델에서 저장되는 값
  - 가중치 값
  - 모델 구조
  - 모델의 훈련 설정(컴파일할 때 전달한 값)
  - 옵티마이저와 (필요하면) 옵티마이저 상태
    - 중지된 지점부터 다시 시작할 수 있습니다

## 사용자 정의 객체
  - SavedModel 포맷
    - 원본 코드 없이 상속 모델과 사용자 정의 층 같은 상용자 정의 객체를 저장할 수 있습니다
    - HDF5와 차이점
      - HDF5가 객체로 모델 구조를 저장
      - SavedModel 실행 그래프로 저장  
        
          
  - HDF5로 저장
    - 객체에 get_config 메서드를 정의하고 선택적으로 from_config 클래스 메서드를 정의
      - get_config(self) : 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환
      - from_config(cls, config) : get_config에서 반환된 설정을 사용해 새로운 객채를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(return cls(**config))
    - 모델을 로드할 때 이 객체를 cusgom_objects 매개변수로 전달
      - 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕셔너리를 매개변수로 제공
      - tf.keras.models.load_model(path, custon_objects={'CustomLayer':CustomLayer})
    - https://www.tensorflow.org/guide/keras/custom_layers_and_models